In [50]:
import json
            
with open('env_var.json') as f:  
    CFG = json.load(f)

# Deploying to Sagemaker through HuggingFace Models

In [41]:
!pip install sagemaker


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import sagemaker
import boto3

In [44]:
from sagemaker.huggingface.model import HuggingFaceModel

role = CFG["iam_role"]

hub = {
	'HF_MODEL_ID': CFG["HF_MODEL_ID"],
	'HF_TASK':CFG["HF_TASK"]
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.17.0',
	pytorch_version='1.10.2',
	py_version='py38',
	env=hub,
	role=role, 
)

In [45]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)

---------!

In [47]:
predictor.predict({
	'inputs': "The Swiss National Bank on Monday reported a loss of 132 billion Swiss francs ($143 billion) for the 2022 financial year, citing preliminary figures.It represents the biggest loss in the central bank’s 116-year history and equates to roughly 18% of Switzerland’s projected gross domestic product of 744.5 billion Swiss francs. Its previous record loss was 23 billion francs in 2015. As a result it will not make its usual payouts to the Swiss government and member states, it said, with payments to its shareholders also set to be affected. In 2021, the bank reported a 26 billion franc profit. Of the losses, 131 billion francs came from its foreign currency positions and 1 billion from its Swiss franc positions amid strong gains made by the franc as investors flocked to the perceived safe haven amid European volatility. Since June 2022, the Swiss franc has been trading above one euro, a level it had previously only briefly touched in 2015 after scrapping its 1.20 peg to the EU’s single currency. Switzerland has historically attempted to rein in the strength of the franc because of its export-heavy economy, though analysts have argued Swiss businesses have been able to remain competitive despite the rising franc due to euro zone inflation. In December, the Swiss National Bank raised interest rates for the third time in 2022, to 1%. That was to counter inflation of 3% — well below the euro zone’s inflation rate, which remains above 10%. The SNB was also impacted last year by losses in its stock and bond portfolio amid the wider market downturn. However, it gained 400 million francs through its gold holdings. Karsten Junius, chief economist at Swiss bank J.Safra Sarasin, told CNBC that the central bank’s losses would not alter its monetary policy and he expected another 100 basis points of hikes, to 2%, this year. “While the SNB will also need some time to rebuild its valuation reserves it will take less time to show profits than in the case of the European Central Bank,” he said, also noting that inflation in Switzerland was closer to its 2% target than in the euro zone. “While both central banks are structurally profitable as they can renumerate their liabilities at a lower rate than the market, the SNB will earn higher market interest this year already while the ECB is stuck with its low yielding bonds in its book and will be unprofitable for many years.”"
})

[{'generated_text': 'Swiss National Bank reports record loss for 2022 financial year'}]

# Using Model

In [68]:
client = boto3.client('sagemaker')
endpoints = client.list_endpoints()

In [69]:
for e in endpoints["Endpoints"]:
    if e["EndpointName"] == CFG["endpoint_name"]:
        name = e["EndpointName"]
        mme_name = name

In [70]:
sess=sagemaker.Session()

predictor=sagemaker.predictor.Predictor(endpoint_name=mme_name, sagemaker_session=sess)
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [71]:
data = {
	"inputs": "The Swiss National Bank on Monday reported a loss of 132 billion Swiss francs ($143 billion) for the 2022 financial year, citing preliminary figures.It represents the biggest loss in the central bank’s 116-year history and equates to roughly 18% of Switzerland’s projected gross domestic product of 744.5 billion Swiss francs. Its previous record loss was 23 billion francs in 2015. As a result it will not make its usual payouts to the Swiss government and member states, it said, with payments to its shareholders also set to be affected. In 2021, the bank reported a 26 billion franc profit. Of the losses, 131 billion francs came from its foreign currency positions and 1 billion from its Swiss franc positions amid strong gains made by the franc as investors flocked to the perceived safe haven amid European volatility. Since June 2022, the Swiss franc has been trading above one euro, a level it had previously only briefly touched in 2015 after scrapping its 1.20 peg to the EU’s single currency. Switzerland has historically attempted to rein in the strength of the franc because of its export-heavy economy, though analysts have argued Swiss businesses have been able to remain competitive despite the rising franc due to euro zone inflation. In December, the Swiss National Bank raised interest rates for the third time in 2022, to 1%. That was to counter inflation of 3% — well below the euro zone’s inflation rate, which remains above 10%. The SNB was also impacted last year by losses in its stock and bond portfolio amid the wider market downturn. However, it gained 400 million francs through its gold holdings. Karsten Junius, chief economist at Swiss bank J.Safra Sarasin, told CNBC that the central bank’s losses would not alter its monetary policy and he expected another 100 basis points of hikes, to 2%, this year. “While the SNB will also need some time to rebuild its valuation reserves it will take less time to show profits than in the case of the European Central Bank,” he said, also noting that inflation in Switzerland was closer to its 2% target than in the euro zone. “While both central banks are structurally profitable as they can renumerate their liabilities at a lower rate than the market, the SNB will earn higher market interest this year already while the ECB is stuck with its low yielding bonds in its book and will be unprofitable for many years.” ",
	'parameters': {
		"num_return_sequences": 5
	}
}

In [72]:
predictor.predict(data)

[{'generated_text': 'Swiss National Bank reports record loss for 2022 financial year'},
 {'generated_text': 'Swiss National Bank reports record loss of 132 billion francs in 2022'},
 {'generated_text': 'Swiss National Bank reports record loss of 132 billion francs for 2022'},
 {'generated_text': 'Swiss National Bank reports record loss of 132 billion Swiss francs in 2022'},
 {'generated_text': 'Swiss National Bank reports record loss of 132 billion francs for 2022 financial year'}]

In [65]:
predictor.predict(data)

[{'generated_text': 'Switzerland’s central bank reports record'}]

# HF API

In [133]:
import json
import requests
api_key = CFG["hf_apikey"]
headers = {"Authorization": f"Bearer {api_key}"}
API_URL = CFG["hf_apiurl"]
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))
data = query(
    {
        "inputs": ["TEST", "The Swiss National Bank on Monday reported a loss of 132 billion Swiss francs ($143 billion) for the 2022 financial year, citing preliminary figures.It represents the biggest loss in the central bank’s 116-year history and equates to roughly 18% of Switzerland’s projected gross domestic product of 744.5 billion Swiss francs. Its previous record loss was 23 billion francs in 2015. As a result it will not make its usual payouts to the Swiss government and member states, it said, with payments to its shareholders also set to be affected. In 2021, the bank reported a 26 billion franc profit. Of the losses, 131 billion francs came from its foreign currency positions and 1 billion from its Swiss franc positions amid strong gains made by the franc as investors flocked to the perceived safe haven amid European volatility. Since June 2022, the Swiss franc has been trading above one euro, a level it had previously only briefly touched in 2015 after scrapping its 1.20 peg to the EU’s single currency. Switzerland has historically attempted to rein in the strength of the franc because of its export-heavy economy, though analysts have argued Swiss businesses have been able to remain competitive despite the rising franc due to euro zone inflation. In December, the Swiss National Bank raised interest rates for the third time in 2022, to 1%. That was to counter inflation of 3% — well below the euro zone’s inflation rate, which remains above 10%. The SNB was also impacted last year by losses in its stock and bond portfolio amid the wider market downturn. However, it gained 400 million francs through its gold holdings. Karsten Junius, chief economist at Swiss bank J.Safra Sarasin, told CNBC that the central bank’s losses would not alter its monetary policy and he expected another 100 basis points of hikes, to 2%, this year. “While the SNB will also need some time to rebuild its valuation reserves it will take less time to show profits than in the case of the European Central Bank,” he said, also noting that inflation in Switzerland was closer to its 2% target than in the euro zone. “While both central banks are structurally profitable as they can renumerate their liabilities at a lower rate than the market, the SNB will earn higher market interest this year already while the ECB is stuck with its low yielding bonds in its book and will be unprofitable for many years.”"],
        "options": {"wait_for_model":True},
        # {}
        }
    )

In [134]:
data

[{'generated_text': 'TEST'},
 {'generated_text': 'The Swiss National Bank on Monday reported the biggest loss in its 116-year history and said it would not make its usual payouts to the Swiss government and member states.'}]